功能概述：
构建完整的数据获取与清洗流水线。负责整合游戏卡池流水数据，爬取 B 站指定视频及其评论，并进行深度的文本清洗，为后续模型训练提供标准语料。

输入：
- 游戏版本记录与外部流水数据 (CSV)
- B 站视频搜索规则配置
输出：
- ./data/character_info.csv (整合后的角色7日流水标签)
- ./data/bilibili_video_data_final.csv (爬取的视频元数据)
- ./data/all_comments_merged.csv (最终清洗、合并并打标的评论语料)

脚本执行流程：

1. 【卡池与流水整合】
   - 读取版本记录与七麦流水数据，对齐时间轴。
   - 剔除异常数据，计算每个角色卡池的“首周（7日）流水”作为预测目标 $y$。

2. 【视频元数据爬取】
   - 实现了 Bilibili WBI 签名算法以绕过接口验证。
   - 根据预设规则（角色PV、走近星穹、头部攻略 UP 主），针对每个角色精准搜索对应的三个特定视频。

3. 【视频评论爬取】
   - 使用[1dyer的b站视频爬虫](https://github.com/1dyer/bilibili-comment-crawler)爬取评论
   - 添加了断点续传机制

4. 【文本清洗与去重】
   - 文本规范化：处理 B 站特有的 [表情包]、Emoji、颜文字等。
   - 繁简转换与符号清洗：统一字符编码，去除无意义符号。
   - 对同一 ID 的重复评论，保留点赞数最高的一条。

5. 【数据合并】
   - 计算每条评论距离视频发布的时间差。
   - 仅保留视频发布后 24 小时内的评论（防止泄漏），输出最终用于训练的合并文件。

In [ ]:
import pandas as pd
from io import StringIO

# --- 1. 加载数据 ---
# 加载版本卡池信息CSV
versions_df = pd.read_csv('./data/崩坏：星穹铁道版本记录_2025-10-27.csv')
revenue_df = pd.read_csv('./data/2023-04-26至2025-10-26 iPhone收入预估.csv')



# --- 2. 构造完整的卡池列表 ---
# 分别处理第一个和第二个卡池
pool1 = versions_df[['版本号', '卡池1', '卡池角色1']].rename(columns={'卡池1': 'start_date', '卡池角色1': 'character'})
pool1['pool_index'] = 1
pool2 = versions_df[['版本号', '卡池2', '卡池角色2']].rename(columns={'卡池2': 'start_date', '卡池角色2': 'character'})
pool2['pool_index'] = 2

# 合并成一个DataFrame
all_pools_df = pd.concat([pool1, pool2])

# --- 3. 数据清洗与预处理 ---
# 转换日期格式
all_pools_df['start_date'] = pd.to_datetime(all_pools_df['start_date'])

# 按开始日期排序
all_pools_df.sort_values('start_date', inplace=True, ignore_index=True)

# --- 4. 计算结束日期和持续时间 ---
# 下一个卡池的开始时间即为当前卡池的结束边界
all_pools_df['end_date'] = all_pools_df['start_date'].shift(-1) - pd.Timedelta(days=1)

# 直接删除没有有效结束日期的行（也就是最后一个卡池）
all_pools_df.dropna(subset=['end_date'], inplace=True)


# 筛选掉指定的不分析的角色
pools_to_analyze_df = all_pools_df[~all_pools_df['character'].isin(['真理医生', '丹恒·腾荒', '无', '白厄', 'Saber'])].copy()

# 计算持续天数
pools_to_analyze_df['duration_days'] = (pools_to_analyze_df['end_date'] - pools_to_analyze_df['start_date']).dt.days + 1


# --- 5. 整合流水数据 ---
# 预处理流水数据
revenue_df.rename(columns={'收入预估~iPhone': 'revenue'}, inplace=True)
revenue_df['日期'] = pd.to_datetime(revenue_df['日期'])
revenue_df.set_index('日期', inplace=True)

# 为每个卡池计算总流水和7日流水
total_revenues = []
seven_day_revenues = [] # <-- 新增一个列表来存储7日流水
for index, row in pools_to_analyze_df.iterrows():
    # --- 计算总流水 (逻辑不变) ---
    total_mask = (revenue_df.index >= row['start_date']) & (revenue_df.index <= row['end_date'])
    total_revenue = revenue_df.loc[total_mask, 'revenue'].sum()
    total_revenues.append(total_revenue)

    # --- 计算7日流水 (新增逻辑) ---
    # 定义7日窗口的结束日期 (开始日期 + 6天)
    seven_day_end_date = row['start_date'] + pd.Timedelta(days=6)
    # 筛选出在7日窗口内的流水数据
    seven_day_mask = (revenue_df.index >= row['start_date']) & (revenue_df.index <= seven_day_end_date)
    seven_day_revenue = revenue_df.loc[seven_day_mask, 'revenue'].sum()
    seven_day_revenues.append(seven_day_revenue)

pools_to_analyze_df['total_revenue'] = total_revenues
pools_to_analyze_df['7day_revenue'] = seven_day_revenues 

# --- 6. 标记性别 ---
# 这是一个让你快速标注的区域。
# 请将下面的角色名列表，根据性别分别填入 female_characters 和 male_characters 列表中。

female_characters = [
    # 在这里粘贴所有女性角色名, 例如:
    "银狼", "卡芙卡", "符玄", "镜流", '托帕&账账', "藿藿", "阮•梅", 
    "黑天鹅", "花火", "黄泉", "知更鸟", "流萤", "翡翠", "云璃", "灵砂",
    "忘归人", "大黑塔", "阿格莱雅", "缇宝", "遐蝶", "风堇", 
    "赛飞儿", "海瑟音", "刻律德菈", "长夜月", "飞霄", "乱破", "Saber"
]

male_characters = [
    # 在这里粘贴所有男性角色名, 例如:
    "刃", "丹恒•饮月", "银枝", "砂金", "波提欧", "椒丘",
     "星期日", "万敌", "罗刹", "那刻夏", "白厄", "无"
]

# 自动构建性别映射字典
gender_map = {character: '女' for character in female_characters}
gender_map.update({character: '男' for character in male_characters})

# 将性别信息映射到DataFrame
pools_to_analyze_df['gender'] = pools_to_analyze_df['character'].map(gender_map)

# (可选) 检查是否有遗漏的角色未被标记
unmapped = pools_to_analyze_df[pools_to_analyze_df['gender'].isnull()]
if not unmapped.empty:
    print("\n[警告] 以下角色没有被分配性别，请检查上面的列表:")
    print(unmapped['character'].unique())

# --- 7. 最终结果展示 ---
# 调整列顺序以便查看
chara_df = pools_to_analyze_df[[
    'character', 'gender', 'start_date', 'end_date', 'duration_days', 'total_revenue','7day_revenue', '版本号', 'pool_index'
]].copy()

# 格式化日期和流水，使其更易读
chara_df['start_date'] = chara_df['start_date'].dt.strftime('%Y-%m-%d')
chara_df['end_date'] = chara_df['end_date'].dt.strftime('%Y-%m-%d')
chara_df['total_revenue'] = chara_df['total_revenue'].apply(lambda x: int(x))

print("各角色卡池持续时间与总流水统计:")
print(chara_df.to_string())
CHARA_FILENAME='./data/character_info.csv'
chara_df.to_csv(CHARA_FILENAME, index=False, encoding='utf-8-sig')

In [ ]:
import requests
import pandas as pd
import time
from hashlib import md5
from functools import reduce
from urllib.parse import urlencode
from datetime import datetime
import re
import numpy as np

MAX_NETWORK_RETRIES = 3 # 底层网络请求失败时的最大重试次数
RETRY_DELAY_SECONDS = 2 # 每次重试前的等待时间
PRECISE_SEARCH_ATTEMPTS = 3 #最大搜索次数
# --- WBI签名算法实现 (这部分代码无需修改) ---
MIXIN_KEY_ENC_TABLE = [
    46, 47, 18, 2, 53, 8, 23, 32, 15, 50, 10, 31, 58, 3, 45, 35, 27, 43, 5, 49,
    33, 9, 42, 19, 29, 28, 14, 39, 12, 38, 41, 13, 37, 48, 7, 16, 24, 55, 40,
    61, 26, 17, 0, 1, 60, 51, 30, 4, 22, 25, 54, 21, 56, 59, 6, 63, 57, 62, 11,
    36, 20, 34, 44, 52
]

def get_mixin_key(orig: str) -> str:
    return reduce(lambda s, i: s + orig[i], MIXIN_KEY_ENC_TABLE, '')[:32]

def wbi_sign_params(params: dict, wbi_keys: dict) -> dict:
    img_key = wbi_keys['img_key']
    sub_key = wbi_keys['sub_key']
    
    mixin_key = get_mixin_key(img_key + sub_key)
    curr_time = round(time.time())
    params['wts'] = curr_time
    
    params = dict(sorted(params.items()))
    params = {
        k: ''.join(filter(lambda ch: ch not in "!*\'()", str(v)))
        for k, v in params.items()
    }
    query = urlencode(params)
    wbi_sign = md5((query + mixin_key).encode()).hexdigest()
    params['w_rid'] = wbi_sign
    
    return params

def get_wbi_keys(headers: dict) -> dict:
    resp = requests.get('https://api.bilibili.com/x/web-interface/nav', headers=headers)
    resp.raise_for_status()
    json_content = resp.json()
    img_url = json_content['data']['wbi_img']['img_url']
    sub_url = json_content['data']['wbi_img']['sub_url']
    wbi_keys = {
        'img_key': img_url.rsplit('/', 1)[1].split('.')[0],
        'sub_key': sub_url.rsplit('/', 1)[1].split('.')[0]
    }
    return wbi_keys
# --- WBI签名算法结束 ---


# --- 爬虫核心函数 (已重构) ---
def _fetch_page_data(params: dict, headers: dict, wbi_keys: dict):
    base_url = "https://api.bilibili.com/x/web-interface/wbi/search/all/v2"
    for attempt in range(MAX_NETWORK_RETRIES):
        try:
            signed_params = wbi_sign_params(params.copy(), wbi_keys)
            response = requests.get(base_url, params=signed_params, headers=headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"    -> 网络请求失败 (尝试 {attempt + 1}/{MAX_NETWORK_RETRIES}): {e}")
            if attempt < MAX_NETWORK_RETRIES - 1:
                time.sleep(RETRY_DELAY_SECONDS)
    return None

def _parse_and_find_first_match(data: dict, up_name: str, char_name: str, video_type: str, pattern: str):
    """解析数据并只返回第一个匹配规则的视频"""
    if not data: return None
    video_list = []
    if 'data' in data and 'result' in data['data']:
        for item in data['data']['result']:
            if item.get('result_type') == 'video': video_list = item.get('data', []); break
    for video in video_list:
        if video.get('author') != up_name: continue
        clean_title = video.get('title', '').replace('<em class="keyword">', '').replace('</em>', '')
        if re.search(pattern, clean_title):
            return {
                'character': char_name, 'video_type': video_type, 'bvid': video.get('bvid'), 
                'title': clean_title, 'author': video.get('author'), 'url': video.get('arcurl'), 
                'play_count': video.get('play'), 'comment_count': video.get('review'), 
                'danmaku_count': video.get('danmaku'), 'like_count': video.get('like'), 
                'favorite_count': video.get('favorites'), 'publish_timestamp': video.get('pubdate'), 
                'duration': video.get('duration'),
            }
    return None # 没有找到匹配项

def find_video_for_rule(char_name, search_name, up_name, rule, formatted_pattern, headers, wbi_keys):
    """为单个规则执行一个多阶段、带重试的搜索级联。"""
    
    # Stage 1: Ultra-Precise Search with Retries
    precise_query = f"{search_name}+{up_name}+{rule['type']}"
    print(f"  -> Stage 1: 精确搜索 '{precise_query}' (最多 {PRECISE_SEARCH_ATTEMPTS} 次)")
    for attempt in range(PRECISE_SEARCH_ATTEMPTS):
        params = {'keyword': precise_query, 'page': 1, 'page_size': 42}
        page_data = _fetch_page_data(params, headers, wbi_keys)
        video = _parse_and_find_first_match(page_data, up_name, char_name, rule['type'], formatted_pattern)
        if video:
            print(f"    -> 在尝试第 {attempt + 1} 次时成功找到!")
            return video
        if attempt < PRECISE_SEARCH_ATTEMPTS - 1:
            print(f"    -> 尝试第 {attempt + 1} 次失败，准备重试...")
            time.sleep(RETRY_DELAY_SECONDS)
            
    # Stage 2: General Search (Fallback)
    general_query = f"{search_name} {up_name}"
    print(f"  -> Stage 2: 降级到常规搜索 '{general_query}'")
    params = {'keyword': general_query, 'page': 1, 'page_size': 42}
    page_data = _fetch_page_data(params, headers, wbi_keys)
    video = _parse_and_find_first_match(page_data, up_name, char_name, rule['type'], formatted_pattern)
    if video:
        print("    -> 成功找到!")
        return video

    # Stage 3: Broad Search (Final Fallback)
    broad_query = search_name
    print(f"  -> Stage 3: 降级到宽泛搜索 '{broad_query}'")
    params = {'keyword': broad_query, 'page': 1, 'page_size': 42}
    page_data = _fetch_page_data(params, headers, wbi_keys)
    video = _parse_and_find_first_match(page_data, up_name, char_name, rule['type'], formatted_pattern)
    if video:
        print("    -> 成功找到!")
        return video
    return None

# ===============================================
# ========== 需要你手动修改的区域 ==========
# ===============================================
MY_COOKIE = "在此处粘贴你的Cookie字符串"
MY_USER_AGENT = "在此处粘贴你的User-Agent字符串"

CHARACTER_ALIASES = {
    "丹恒•饮月": "饮月",
    "阮•梅": "阮梅",
    "托帕&账账": "托帕",
    # 未来可在此添加更多别名
}

# 2. 视频查询规则配置 (核心)
#    - {CHAR} 将被替换为角色的全名或别名
#    - pattern 是一个正则表达式
SEARCH_CONFIG = {
    "崩坏星穹铁道": [
        {
            "type": "角色PV",
            "pattern": r"《崩坏：星穹铁道》.*?{CHAR}.*?角色PV"
        },
        {
            "type": "走近星穹",
            "pattern": r"《崩坏：星穹铁道》.*?走近星穹.*?「{CHAR}"
        }
    ],
    "卡特亚": [
        {
            "type": "综合",
            "pattern": r"{CHAR}.*?综合"
        }
    ]

}
characters_to_search = list(chara_df['character'])
# ===============================================

if __name__ == '__main__':
    if "在此处粘贴你的Cookie字符串" in MY_COOKIE:
        print("错误：请先在脚本中填入你自己的 MY_COOKIE 和 MY_USER_AGENT！"); exit()
    
    CSV_FILENAME = "./data/bilibili_video_data_final.csv"

    already_found = set()
    
    try:
        existing_df = pd.read_csv(CSV_FILENAME)
        # 使用zip和set comprehension高效创建查询集合
        already_found = set(zip(existing_df['character'], existing_df['author'], existing_df['video_type']))
        print(f"成功读取已有的 {len(existing_df)} 条记录 from '{CSV_FILENAME}'\n")
    except FileNotFoundError:
        existing_df = pd.DataFrame() # 如果文件不存在，创建一个空的DataFrame
        print(f"未找到 '{CSV_FILENAME}'。将从头开始爬取。\n")
    # --- 断点续爬逻辑结束 ---

    request_headers = { 'User-Agent': MY_USER_AGENT, 'Cookie': MY_COOKIE, 'Referer': 'https://search.bilibili.com/' }
    
    print("正在获取最新的WBI密钥..."); 
    try: wbi_keys = get_wbi_keys(request_headers); print("WBI密钥获取成功!\n")
    except Exception as e: print(f"致命错误：获取WBI密钥失败，程序将退出。错误信息: {e}"); exit()
        
    newly_found_videos = [] # 只存储本次新发现的视频

    for char_name in characters_to_search:
        search_name = CHARACTER_ALIASES.get(char_name, char_name)
        alias = CHARACTER_ALIASES.get(char_name)
        pattern_char_name = f"({re.escape(char_name)}|{re.escape(alias)})" if alias else re.escape(char_name)
        
        for up_name, rules in SEARCH_CONFIG.items():
            for rule in rules:
                lookup_key = (char_name, up_name, rule['type'])
                if lookup_key in already_found:
                    print(f"--- 已存在记录，跳过 (UP: '{up_name}', Char: '{char_name}', Type: '{rule['type']}') ---")
                    continue # 跳过本次循环，进行下一个
                
                print(f"--- 开始为 (UP主: '{up_name}', 角色: '{char_name}', 类型: '{rule['type']}') 查找视频 ---")
                
                formatted_pattern = rule['pattern'].format(CHAR=pattern_char_name)
                found_video = find_video_for_rule(char_name, search_name, up_name, rule, formatted_pattern, request_headers, wbi_keys)
                
                if found_video:
                    newly_found_videos.append(found_video)
                else:
                    print(f"[警告] 所有搜索阶段均失败，未能找到 (UP: '{up_name}', Char: '{char_name}', Type: '{rule['type']}') 的视频。")
                
                print("-" * 70)
                time.sleep(np.random.uniform(1,3))

    if newly_found_videos or not existing_df.empty:
        new_df = pd.DataFrame(newly_found_videos)
        # 合并旧数据和新数据
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        
        # 对合并后的完整数据进行后处理
        combined_df.drop_duplicates(subset=['character', 'author', 'video_type'], inplace=True, keep='last')
        
        # 只有在timestamp列存在时才进行转换
        if 'publish_timestamp' in combined_df.columns:
            combined_df['publish_date'] = pd.to_datetime(combined_df['publish_timestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')

        sorted_df = combined_df.sort_values(by=['character', 'author', 'video_type']).reset_index(drop=True)

        print("\n=============== 任务完成 ================")
        print(f"本次运行新增了 {len(newly_found_videos)} 条记录。")
        print(f"总共 {len(sorted_df)} 条记录已保存到 '{CSV_FILENAME}'")
        
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_colwidth', 40)
        
        display_columns = ['character', 'author', 'video_type', 'title', 'play_count', 'comment_count', 'bvid']
        print(sorted_df[display_columns])
        
        sorted_df.to_csv(CSV_FILENAME, index=False, encoding='utf-8-sig')
    else:
        print("\n任务完成，但未能找到任何符合查询规则的视频。")

In [ ]:
import re
import requests
import json
from urllib.parse import quote
import pandas as pd
import hashlib
import urllib
import time
import csv
import pprint
import numpy as np
import os


# 通过bv号，获取视频的oid
def get_information(bv):
    resp = requests.get(f"https://www.bilibili.com/video/{bv}/?p=14&spm_id_from=pageDriver&vd_source=cd6ee6b033cd2da64359bad72619ca8a",headers=HEADER)
    # 提取视频oid
    obj = re.compile(f'"aid":(?P<id>.*?),"bvid":"{bv}"')
    oid = obj.search(resp.text).group('id')

    # 提取视频的标题
    obj = re.compile(r'<title data-vue-meta="true">(?P<title>.*?)</title>')
    try:
        title = obj.search(resp.text).group('title')
    except:
        title = "未识别"

    return oid,title

# MD5加密
def md5(code):
    MD5 = hashlib.md5()
    MD5.update(code.encode('utf-8'))
    w_rid = MD5.hexdigest()
    return w_rid

# 轮页爬取
def start(bv, oid, pageID, count, csv_writer, is_second):
    # 参数
    mode = 3   # 为2时爬取的是最新评论，为3时爬取的是热门评论
    plat = 1
    type = 1  
    web_location = 1315875

    # 获取当下时间戳
    wts = int(time.time())
    
    # 如果不是第一页
    if pageID != '':
        pagination_str = '{"offset":"%s"}' % pageID
        code = f"mode={mode}&oid={oid}&pagination_str={urllib.parse.quote(pagination_str, safe='')}&plat={plat}&type={type}&web_location={web_location}&wts={wts}" + 'ea1db124af3c7062474693fa704f4ff8'
        w_rid = md5(code)
        url = f"https://api.bilibili.com/x/v2/reply/wbi/main?oid={oid}&type={type}&mode={mode}&pagination_str={urllib.parse.quote(pagination_str, safe=':')}&plat=1&web_location=1315875&w_rid={w_rid}&wts={wts}"
    
    # 如果是第一页
    else:
        pagination_str = '{"offset":""}'
        code = f"mode={mode}&oid={oid}&pagination_str={urllib.parse.quote(pagination_str, safe='')}&plat={plat}&seek_rpid=&type={type}&web_location={web_location}&wts={wts}" + 'ea1db124af3c7062474693fa704f4ff8'
        w_rid = md5(code)
        url = f"https://api.bilibili.com/x/v2/reply/wbi/main?oid={oid}&type={type}&mode={mode}&pagination_str={urllib.parse.quote(pagination_str, safe=':')}&plat=1&seek_rpid=&web_location=1315875&w_rid={w_rid}&wts={wts}"
    

    comment = requests.get(url=url, headers=HEADER).content.decode('utf-8')
    comment = json.loads(comment)
    if comment.get('code') != 0:
        print(f"API返回错误！ Code: {comment.get('code')}, Message: {comment.get('message')}")
        print("这通常意味着Cookie失效、WBI签名错误或IP被限制。")
        print(comment)


    # ==================== 安全地访问数据 ====================
    # 经过上面的检查，现在可以比较安全地假设'data'键存在了
    # 但为了绝对安全，我们还是用 .get() 方法
    replies_list = comment.get('data', {}).get('replies')
    if not replies_list:
        print("当前页没有评论数据或已到达末页。")

    for reply in comment['data']['replies']:
        # 评论数量+1
        count += 1

        if count % 1000 ==0:
            time.sleep(20)

        # 上级评论ID
        parent=reply["parent"]
        # 评论ID
        rpid = reply["rpid"]
        # 用户ID
        uid = reply["mid"]
        # 用户名
        name = reply["member"]["uname"]
        # 用户等级
        level = reply["member"]["level_info"]["current_level"]
        # 性别
        sex = reply["member"]["sex"]
        # 头像
        avatar = reply["member"]["avatar"]
        # 是否是大会员
        if reply["member"]["vip"]["vipStatus"] == 0:
            vip = "否"
        else:
            vip = "是"
        # IP属地
        try:
            IP = reply["reply_control"]['location'][5:]
        except:
            IP = "未知"
        # 内容
        context = reply["content"]["message"]
        # 评论时间
        reply_time = pd.to_datetime(reply["ctime"], unit='s')
        # 相关回复数
        try:
            rereply = reply["reply_control"]["sub_reply_entry_text"]
            rereply = int(re.findall(r'\d+', rereply)[0])
        except:
            rereply = 0
        # 点赞数
        like = reply['like']


        # 写入CSV文件
        csv_writer.writerow([count, parent, rpid, uid, name, level, sex, context, reply_time, rereply, like, IP, vip])

        # 二级评论(如果开启了二级评论爬取，且该评论回复数不为0，则爬取该评论的二级评论)
        if is_second and rereply !=0:
            for page in range(1,(rereply-1)//10+2):
                second_url=f"https://api.bilibili.com/x/v2/reply/reply?oid={oid}&type=1&root={rpid}&ps=10&pn={page}&web_location=333.788"
                second_comment=requests.get(url=second_url,headers=HEADER).content.decode('utf-8')
                second_comment=json.loads(second_comment)
                # pprint.pprint(second_comment, width=80)
                if second_comment['data']['replies'] is None:
                    print('find None')
                    # print(second_url)
                    continue
                for second in second_comment['data']['replies']:
                    # 评论数量+1
                    count += 1
                    # 上级评论ID
                    parent=second["parent"]
                    # 评论ID
                    second_rpid = second["rpid"]
                    # 用户ID
                    uid = second["mid"]
                    # 用户名
                    name = second["member"]["uname"]
                    # 用户等级
                    level = second["member"]["level_info"]["current_level"]
                    # 性别
                    sex = second["member"]["sex"]
                    # 是否是大会员
                    if second["member"]["vip"]["vipStatus"] == 0:
                        vip = "否"
                    else:
                        vip = "是"
                    # IP属地
                    try:
                        IP = second["reply_control"]['location'][5:]
                    except:
                        IP = "未知"
                    # 内容
                    context = second["content"]["message"]
                    # 评论时间
                    reply_time = pd.to_datetime(second["ctime"], unit='s')
                    # 相关回复数
                    try:
                        rereply = second["reply_control"]["sub_reply_entry_text"]
                        rereply = re.findall(r'\d+', rereply)[0]
                    except:
                        rereply = 0
                    # 点赞数
                    like = second['like']


                    # 写入CSV文件
                    csv_writer.writerow([count, parent, second_rpid, uid, name, level, sex, context, reply_time, rereply, like, IP, vip])

                time.sleep(np.random.uniform(0.6,1.2))
            


    # 下一页的pageID
    try:
        next_pageID = comment['data']['cursor']['pagination_reply']['next_offset']
    except:
        next_pageID = 0

    # 判断是否是最后一页了
    if next_pageID == 0:
        print(f"评论爬取完成！总共爬取{count}条。")
        return bv, oid, next_pageID, count, csv_writer,is_second
    # 如果不是最后一页，则停0.5s（避免反爬机制）
    else:
        time.sleep(np.random.uniform(0.6,1.2))
        print(f"当前爬取{count}条。")
        return bv, oid, next_pageID, count, csv_writer,is_second

HEADER={
        "Cookie":MY_COOKIE,
        "User-Agent": MY_USER_AGENT
}

if __name__ == "__main__":

    
    CSV_FILENAME = "./data/bilibili_video_data_final.csv"
    video_df = pd.read_csv(CSV_FILENAME)
    for i in range(len(video_df)):
        bv = video_df['bvid'][i]
        # 获取视频oid和标题
        oid,title = get_information(bv)
        # 评论起始页（默认为空）
        next_pageID = ''
        # 初始化评论数量
        count = 0
        # --- 以下是实现断点续爬的核心修改 ---
        
        comment_csv_path = f'./data/{bv}_comment.csv'
        progress_json_path = f'./data/{bv}_progress.json'

        next_pageID = ''
        count = 0
        file_open_mode = 'w'
        write_header = True
        is_second = True

        # 检查是否存在进度文件
        if os.path.exists(progress_json_path):
            try:
                with open(progress_json_path, 'r') as f:
                    progress_data = json.load(f)
                    next_pageID = progress_data.get('next_pageID', '')
                    count = progress_data.get('count', 0)
                
                file_open_mode = 'a' # 如果恢复，使用追加模式
                write_header = False  # 不再写入表头
                print(f"[{bv}] 检测到进度文件，将从 {count} 条评论后继续。")
            except Exception as e:

                print(f"[{bv}] 进度文件读取失败: {e}。将从头开始爬取。")
                # 如果进度文件损坏，则重置为默认值
                next_pageID, count, file_open_mode, write_header = '', 0, 'w', True
        elif os.path.exists(comment_csv_path):
            print(f'删除{bv}，重新爬取')
            os.remove(comment_csv_path)
            

        try:
            with open(comment_csv_path, mode=file_open_mode, newline='', encoding='utf-8-sig') as file:
                csv_writer = csv.writer(file)
                if write_header:
                    csv_writer.writerow(['序号', '上级评论ID', '评论ID', '用户ID', '用户名', '用户等级', '性别', '评论内容', '评论时间', '回复数', '点赞数', 'IP属地', '是否是大会员'])

                # 使用一个标志位确保循环至少能进入一次
                is_first_loop = True
                while next_pageID != 0 or is_first_loop:
                    is_first_loop = False # 进入后立即置为False
                    
                    # 执行爬取
                    bv, oid, next_pageID, count, csv_writer, is_second = start(bv, oid, next_pageID, count, csv_writer, is_second)

                    # 如果还有下一页，保存当前进度
                    if next_pageID != 0:
                        with open(progress_json_path, 'w') as f:
                            json.dump({'next_pageID': next_pageID, 'count': count}, f)
            
            # 任务成功完成后，删除进度文件
            if os.path.exists(progress_json_path):
                os.remove(progress_json_path)
            
            print(f'任务{bv}已完成！')

        except Exception as e:
            # 如果发生任何错误（如网络中断、键盘中断），程序会在这里退出
            # 此时进度文件会保留下来，供下次运行使用
            print(f"任务 {bv} 意外中断: {e}")
            print(f"当前进度已保存至 {progress_json_path}。下次运行时将自动恢复。")


In [ ]:
import pandas as pd
from IPython.display import display
import re
import os
import emoji
import opencc
from cleantext import clean
import ftfy
import math
import unicodedata as ucd
import regex 
import emoji
import opencc

# =========================
# 预编译/单例资源
# =========================

_OPENCC_T2S = opencc.OpenCC("t2s")  # 复用 OpenCC 实例

# [xxx_yyy]（至少一个下划线）
_PAT_BILI_BRACKET = re.compile(r'\[([^\[\]]*?[_-][^\[\]]*?)\]')

# 行首“回复/回覆/回復 … ：/:”前缀
_PAT_REPLY_PREFIX = re.compile(r'(?s)^\s*(?:回复|回覆|回復)\s*@?.*?[：:]\s*')

# ᐠ( ᑒ )ᐟ → [欢呼]（括号内空格宽松）
_PAT_CHEER = re.compile(r"ᐠ\(\s*ᑒ\s*\)ᐟ")

# 连续句号（全/半角，允许空白夹杂）→ 单个 “…”
_PAT_ELLIPSIS = re.compile(r'[。\.．·・](?:\s*[。\.．·・]){1,}')

# Default-Ignorable 与 Unicode 空白（clean 阶段用）
_PAT_DEFAULT_IGNORABLE = regex.compile(r'\p{Default_Ignorable_Code_Point}+')
_PAT_ZS = regex.compile(r'\p{Z}+')

# 直删 kaomoji 模板 → 构造“空格宽松”的交替正则
zap = re.compile(r'[\u200B-\u200D\u2060\u00AD\uFEFF]')
with open('emoji_list.txt','r',encoding='utf-8') as f:
    emoji_list = [zap.sub('', line).rstrip('\n') for line in f]


_KAOMOJI_RAW = [
    "→_→",
    "(♡ ὅ ◡ ὅ )ʃ♡",
    "(/^▽^)/",
    "(>y<)",
    "(/TДT)/",
    "(/∇/)",
    "(ノ)`ω´(ヾ)",
    "₍˄·͈༝·͈˄*₎◞ ̑!",
    "_(:D)∠)_",
    "(¦3【▓】",
    "ಥ_ಥ",
    "(T▽T)",
    "ᕙ(o‸o)ᕗ",
    "੧ᐛ੭",
    "(▼へ▼メ)",
    "(^O^)y",
    "=-=",
    "(∠・ω< )⌒★"
]
_KAOMOJI_RAW.extend(emoji_list)
_KAOMOJI_RAW= list(set(_KAOMOJI_RAW))
def _compile_kaomoji_union(raw_list):
    parts = []
    for s in raw_list:
        p = re.escape(s)
        # 模板中的（一个或多个）空格 → \s*（空格宽松）
        p = re.sub(r'\\\s+', r'\\s*', p)
        parts.append(p)
    if not parts:
        return None
    return re.compile(r'(?:' + '|'.join(parts) + r')')
_PAT_KAOMOJI_UNION = _compile_kaomoji_union(_KAOMOJI_RAW)

FTZ = (
    "_light_skin_tone",
    "_medium_light_skin_tone",
    "_medium_skin_tone",
    "_medium_dark_skin_tone",
    "_dark_skin_tone",
)

# 严格白名单（首项为 canonical）
alias_groups = [
    ["爱心", "red_heart", "heart", "heart_suit", "心"],
    ["心碎", "broken_heart", ],
    ["点赞", "thumbs_up", "赞", "赞了"],
    ["ok", "OK", "Ok", "OK_button", "ok_button"],
    ["保佑", "folded_hands", "祈祷", "祈愿"],
    ["鼓掌", "clapping_hands",],
    ["胜利", "victory_hand", "耶"],
    ["玫瑰","rose",],
    ["问号", "red_question_mark", "?", "黑人问号"],
    ["飞吻","face_blowing_a_kiss",],
    ["吃瓜", "吃瓜ing"],
]

# 建立严格映射（键统一为小写）
alias_map = {}
for group in alias_groups:
    canon = group[0].lower()
    for name in group:
        alias_map[name.lower()] = canon

# 对单个方括号内容做规范化
def canon_name(name: str) -> str:
    x = name.lower()
    # 1) 先处理肤色：如果命中，去色得到 base
    had_tone = False
    for suf in FTZ:
        if x.endswith(suf):
            x = x[: -len(suf)]
            had_tone = True
            break
    # 2) 严格白名单同义映射
    if x in alias_map:
        return alias_map[x]
    # 3) 仅肤色变体：去色后直接用 base
    if had_tone:
        return x
    # 4) 否则保持原样（不改变大小写/写法）
    return name

# 将字符串中每个 [...] 替换为规范名
_PAT_EMOJI = re.compile(r"\[([^\[\]]+)\]")

def merge_emoji(text: str) -> str:
    return _PAT_EMOJI.sub(lambda m: f"[{canon_name(m.group(1))}]", text)

# 括号符号块的“左右标记”，保持原集合（为兼容性预先按长度降序）
_LEFT_TOKENS  = sorted(["(", "^", "•", "|", ">", "\\","━","ᗜ","⌓","₎","ノ","ヾ","୧","✪","╯","ʕ"],
                       key=len, reverse=True)
_RIGHT_TOKENS = sorted([")", "^", "•", "|", "<", "/","━","ᗜ","⌓","₎","ノ","୨","✪","╰","ʔ","】"],
                       key=len, reverse=True)

# clean 阶段的翻译与白名单（与现有逻辑保持一致）
_ALLOWED_SCRIPTS = ("Han", "Hiragana", "Katakana", "Hangul", "Latin")
_ALLOW_PUNCT = set(r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""") | set(
    "，。！？：；、（）《》〈〉“”‘’【】〖〗「」『』〔〕—…·・"
)
_ALLOW_SYMBOL = set("°×÷￥¥€…−")
_TRANSLATE_MAP = {
    0x2013: ord('-'), 0x2014: ord('-'), 0x2212: ord('-'),
    ord('『'): ord('「'), ord('』'): ord('」'),
    ord('〖'): ord('【'), ord('〗'): ord('】'),
    ord('〈'): ord('《'), ord('〉'): ord('》'),
    ord('〔'): ord('('), ord('〕'): ord(')'),
    ord('·'): ord('・'),
}

# =========================
# 统一的字符分类（保持既有语义）
# =========================

def _is_wordlike_basic(ch: str) -> bool:
    """与现有 _is_en/_is_digit/_is_cn_basic/_is_japanese/_is_korean 一致的“文字类”判定。
    - 英文仅 ASCII A–Z/a–z
    - 数字仅 ASCII 0–9
    - 中文仅基本区 U+4E00–U+9FFF
    - 日/韩按原有区间
    """
    if 'A' <= ch <= 'Z' or 'a' <= ch <= 'z':
        return True
    if '0' <= ch <= '9':
        return True
    if '\u4e00' <= ch <= '\u9fff':
        return True
    if ('\u3040' <= ch <= '\u309f') or ('\u30a0' <= ch <= '\u30ff') \
       or ('\u31f0' <= ch <= '\u31ff') or ('\uff65' <= ch <= '\uff9f'):
        return True  # Japanese
    if ('\u1100' <= ch <= '\u11ff') or ('\u3130' <= ch <= '\u318f') \
       or ('\ua960' <= ch <= '\ua97f') or ('\ud7b0' <= ch <= '\ud7ff') \
       or ('\uac00' <= ch <= '\ud7af') or ('\uffa0' <= ch <= '\uffdc'):
        return True  # Korean
    return False

def _is_symbol(ch: str, *, punct_as_symbol: bool) -> bool:
    """统一的 symbol 判定入口：
    - 空白与方括号 '[]' 永远不算 symbol（**硬编码保护方括号**）
    - 其它是否为 symbol：非“文字类”则认为是 symbol；
      若 punct_as_symbol=True，则 Unicode 标点也视为 symbol（括号表情检测用）
    """
    if ch.isspace() or ch in '[]':   # 硬编码保护方括号
        return False
    if _is_wordlike_basic(ch):
        return False
    # 标点是否计为 symbol：括号检测需要 True，其余场景为 False
    if not punct_as_symbol and ucd.category(ch).startswith('P'):
        return False
    return True

# “后者优先”区间去重（包含关系保留新来的）
def _later_wins_add(existing, new_span):
    s0, s1, frag = new_span
    kept = []
    for e0, e1, efrag in existing:
        if (e0 >= s0 and e1 <= s1) or (s0 >= e0 and s1 <= e1):
            continue
        kept.append((e0, e1, efrag))
    kept.append((s0, s1, frag))
    return kept

# =========================
# 子步骤：括号符号块删除（保持 ratio_thr=1/2）
# =========================

def _strip_emoticon_with_boundaries(text: str, ratio_thr: float = 1/2):
    """查找左右标记包裹、内部“括号用符号”占比>ratio_thr 的片段；连同两侧连续符号/空格一起替换为单空格。"""
    def _starts_with_any(s, i, toks):
        for t in toks:  # 已按长度降序
            if s.startswith(t, i):
                return t
        return None

    spans = []
    s = text
    changed = True
    while changed:
        changed = False
        i = 0
        n = len(s)
        while i < n:
            lt = _starts_with_any(s, i, _LEFT_TOKENS)
            if not lt:
                i += 1; continue
            l0, l1 = i, i + len(lt)

            # 找第一个右标记
            j, rtok = l1, None
            while j < n:
                rt = _starts_with_any(s, j, _RIGHT_TOKENS)
                if rt: rtok = rt; break
                j += 1
            if not rtok:
                i += 1; continue
            r0, r1 = j, j + len(rtok)

            inner = s[l1:r0]
            non_space_len = sum(1 for ch in inner if not ch.isspace())
            if non_space_len > 0:
                sym_cnt = sum(_is_symbol(ch, punct_as_symbol=True) for ch in inner if not ch.isspace())
                if (sym_cnt / non_space_len) > ratio_thr:
                    # 向两侧扩展：连续“括号场景的符号”或空格
                    L = l0
                    while L > 0 and (_is_symbol(s[L-1], punct_as_symbol=True) or s[L-1].isspace()):
                        L -= 1
                    R = r1
                    while R < n and (_is_symbol(s[R], punct_as_symbol=True) or s[R].isspace()):
                        R += 1
                    frag = s[L:R]
                    spans = _later_wins_add(spans, (L, R, frag))
                    s = s[:L] + " " + s[R:]
                    n = len(s); i = L + 1; changed = True
                    continue
            i = l0 + 1
    return s, spans

# =========================
# 子步骤：统一的 Unicode 清洗（保持现有 clean_unicode_text 行为）
# =========================

def _script_of(ch: str) -> str | None:
    for sc in _ALLOWED_SCRIPTS:
        if regex.match(fr'\p{{Script={sc}}}$', ch):
            return sc
    return None

def _clean_unicode_text(text: str) -> str:
    s = ucd.normalize('NFKC', text)
    # 1) 删除 Default Ignorable
    s = _PAT_DEFAULT_IGNORABLE.sub('', s)
    # 2) Unicode 空白折叠
    s = _PAT_ZS.sub(' ', s).strip()
    # 3) 翻译映射（破折号/书名号等）
    s = s.translate(_TRANSLATE_MAP)

    out = []
    prev_is_space = False
    for ch in s:
        if ch.isspace():
            if not prev_is_space:
                out.append(' ')
                prev_is_space = True
            continue
        prev_is_space = False

        cat = ucd.category(ch)   # 'Ll','Lo','Nd','Ps','Pe','Po','Sm',...
        sc  = _script_of(ch)

        # 字母（限定脚本）
        if cat[0] == 'L' and sc in _ALLOWED_SCRIPTS:
            out.append(ch); continue
        # 数字（仅 ASCII）
        if cat == 'Nd' and ('0' <= ch <= '9'):
            out.append(ch); continue
        # 允许的标点
        if ch in _ALLOW_PUNCT:
            out.append(ch); continue
        # 少量符号白名单
        if ch in _ALLOW_SYMBOL:
            out.append(ch); continue
        # 组合附加符号（Mn）默认删除；其他 S*（Symbol）默认删除
        if cat == 'Mn' or cat[0] == 'S':
            continue
        # 其余丢弃
    return ''.join(out).strip()

# =========================
# 子步骤：折叠重复“符号”
# =========================

def _fold_repeated_symbols(s: str) -> str:
    if not s: return s
    out, prev, run = [], None, 0
    for ch in s:
        if prev is None:
            prev, run = ch, 1; continue
        if ch == prev:
            run += 1
        else:
            if _is_symbol(prev, punct_as_symbol=False) and run >= 2:
                out.append(prev)          # 符号连串≥2 → 1
            else:
                out.append(prev * run)    # 非符号或未重复 → 原样
            prev, run = ch, 1
    if prev is not None:
        if _is_symbol(prev, punct_as_symbol=False) and run >= 2:
            out.append(prev)
        else:
            out.append(prev * run)
    return ''.join(out)

# =========================
# 主入口：单条评论文本清洗（保持现有顺序与效果）
# =========================
_PAT_UNIT_4PLUS_IGNSPACE_NODIG_TWS = re.compile(r'([^\s\d]+?)(?:\s*\1){3,}(\s*)')
_PAT_BILI_TUODAN_PREFIX = re.compile(r'\[(?:脱单)+([^\[\]]+?)\]')
_PAT_CIALLO_OUTSIDE = regex.compile(
    r'\[[^\[\]]+\](*SKIP)(*F)|(?<![A-Za-z0-9_])(?:Ciallo|ciallo)(?![A-Za-z0-9_])'
)


    


# —— A) (x)/(√) 翻译（在删除表情之前用）——
_CHECK_CHARS = "√✓✔✅☑🗸"
_CROSS_CHARS = "xX✗✘✕×❌"
_PAT_PAREN_CROSS_TO_BAD = re.compile(r'\(\s*([' + re.escape(_CROSS_CHARS) + r'])\s*\)')
_PAT_PAREN_CHECK_TO_OK  = re.compile(r'\(\s*([' + re.escape(_CHECK_CHARS) + r'])\s*\)')
_PAT_STANDALONE_CHECK   = re.compile(r'[' + re.escape(_CHECK_CHARS) + r']')

# —— C) 时间戳 mm:ss / m:ss —— 仅 00–59 秒，分钟 0–59；左右不粘数字
_PAT_TIMESTAMP = re.compile(r'(?<!\d)(?:[0-5]?\d)[：:][0-5]\d(?!\d)')

# —— D) URL 与 BV 号 —— 先 URL、后 BV（独立 token）
_PAT_URL = re.compile(r'(?:(?<!\w)(?:https?|ftp)://|(?<!\w)www\.)[^\s<>"“”‘’]+')
_PAT_BV  = re.compile(r'(?<![A-Za-z0-9])BV[A-Za-z0-9]{10}(?![A-Za-z0-9])')

# —— B) 激活码（C……N，刚好长度10，独立 token）——
_PAT_ACTIVATION = re.compile(r'(?<![A-Za-z0-9])C[A-Z0-9]{8}N(?![A-Za-z0-9])')

def shrink_runs_to_three(s: str) -> str:
    if not s:
        return s
    prev = None
    while prev != s:  # 用于处理像 "aaaab"*4 -> "aaab"*3 的二次收敛
        prev = s
        s = _PAT_UNIT_4PLUS_IGNSPACE_NODIG_TWS.sub(r'\1\1\1\2', s)
    return s

def clean_comment_text_pipeline(text: object) -> str:
    """
    完整文本清洗流水线（单条字符串）——保持与现有代码“可观察结果”一致：
      1) 规范化 B站方括号表情：[xxx_yyy] → 若末尾为 _金箍 则去该后缀，否则保留最后一段：[a_b_c]→[c]
      2) emoji → [alias]（emoji.demojize；保留方括号本体）
      3) OpenCC t2s（繁→简）
      4) 初步空白统一：将 \r \n \t → 空格，并折叠空白
      5) 删除行首“回复/回覆/回復 … ：/:”前缀
      6) 符号与颜文字处理（与现有 preprocess_symbol_squeeze 等价）：
         6.1) 直删若干 kaomoji 模板（宽松空格；后者优先去重）
         6.2) 再次 NFKC（保持现有两次 NFKC 的顺序，以维持完全一致结果）
         6.3) 'ᐠ( ᑒ )ᐟ' → '[欢呼]'         
         6.4) 括号符号块（内部符号占比>1/2）连同两侧相连符号/空格 → 单空格
         6.5) _clean_unicode_text：再次 NFKC + Default-Ignorable 删除 + 脚本/标点/符号白名单过滤
         6.4) 连续句号（全/半角，允许空白夹杂）→ 单个“…”， 再次删除多余空格
         6.7) 折叠连续相同“符号”为 1 个
    重要：方括号字符 '[' 和 ']' **始终**被硬编码为不参与任何删除/折叠判定（但括号内的文字仍按既有规则改写）。
    """
    # —— 入参标准化（与现有行为一致，NaN/None → ""）
    if text is None:
        s = ""
    elif isinstance(text, float) and math.isnan(text):
        s = ""
    else:
        s = str(text)
        
    # 1) B站方括号表情裁剪
    def _bili_sub(m: re.Match) -> str:
        inner = m.group(1)
        if inner.endswith("_金箍"):
            new_inner = inner[:-len("_金箍")]
        else:
            new_inner = inner.rsplit("_", 1)[-1] 
            new_inner = new_inner.rsplit("-", 1)[-1]
        low = inner.lower()
        if "问号" in inner:
            return "[问号]"
        if "点赞" in inner:
            return "[点赞]"
        if "ciallo" in low:
            return "[Ciallo]"
        return f"[{new_inner}]"

    s = _PAT_BILI_BRACKET.sub(_bili_sub, s)
    s = _PAT_BILI_TUODAN_PREFIX.sub(lambda m: f"[{m.group(1)}]", s)
    s = _PAT_CIALLO_OUTSIDE.sub('[Ciallo]', s)
    s = merge_emoji(s)

    # [新增 - A] 先把 (x) 系 和 √ 系翻译成标签（在删除表情之前）
    s = _PAT_PAREN_CROSS_TO_BAD.sub("[错]", s)   # (x / × / ✗ / ✘ / ✕ / ❌) → [错]
    s = _PAT_PAREN_CHECK_TO_OK.sub("[对]", s)    # (√ / ✓ / ✔ / ✅ / ☑ / 🗸) → [对]
    s = _PAT_STANDALONE_CHECK.sub("[对]", s)     # 独立的 √ 系符号 → [对]
    # 2) emoji → [alias]
    s = emoji.demojize(s, language="en", delimiters=("[", "]"))

    # 3) 繁→简
    s = _OPENCC_T2S.convert(s)

    # 4) 初步空白统一（等价于你原先的 cleantext 用途）
    s = re.sub(r'[\r\n\t]+', ' ', s)
    s = _PAT_ZS.sub(' ', s).strip()

    # [新增 - D] URL → [链接]（优先整段 URL）
    s = _PAT_URL.sub("[链接]", s)
    # [新增 - D] BV 号 → [链接]（独立 token）
    s = _PAT_BV.sub("[链接]", s)

    # [新增 - C] 时间戳（m:ss / mm:ss）→ [时间戳]
    s = _PAT_TIMESTAMP.sub("[时间戳]", s)

    # [新增 - B] 激活码（C********N，长度10）→ 删除
    s = _PAT_ACTIVATION.sub("", s)

    # 5) 删除行首“回复 … ：/:”前缀（只清行首）
    s = _PAT_REPLY_PREFIX.sub('', s, count=1)

    # 6.1) 直删 kaomoji 模板（宽松空格；后者优先）
    if _PAT_KAOMOJI_UNION:
        spans = [(m.start(), m.end(), m.group(0)) for m in _PAT_KAOMOJI_UNION.finditer(s)]
        kept = []
        for sp in spans:
            kept = _later_wins_add(kept, sp)
        for st, ed, _ in sorted(kept, key=lambda x: x[0], reverse=True):
            s = s[:st] + "" + s[ed:]

    # 6.2) NFKC
    s = ucd.normalize('NFKC', s)
    s = ftfy.fix_text(s)

    # 6.3) 特例映射
    s = _PAT_CHEER.sub("[欢呼]", s)



    # 6.4) 括号符号块删除（内部符号>1/2） 
    s, _ = _strip_emoticon_with_boundaries(s, ratio_thr=1/2)

    # 6.5) 统一 Unicode 清洗（NFKC + DefaultIgnorable + 白名单过滤）
    s = _clean_unicode_text(s)

    # 6.6) 连续句号 → “…” 再次删除多余空格
    s = _PAT_ELLIPSIS.sub("…", s)
    s = _PAT_ZS.sub(' ', s).strip()
    
    
    # 6.7) 折叠重复“符号”（[]/空白/文字类不参与）
    s = _fold_repeated_symbols(s)
    s = shrink_runs_to_three(s)

    # _PAT_BRACKET_TOKEN = re.compile(r'\[[^\[\]]+?\]')
    # for tok in _PAT_BRACKET_TOKEN.findall(s):
    #     emote_counter[tok] = emote_counter.get(tok, 0) + 1
    return s

# 列级便捷封装（可选）
def clean_comment_series(series):
    """对 pandas Series 应用文本清洗。调用方负责确保 dtype 为 str 或可被 str() 化。"""
    return series.map(clean_comment_text_pipeline)

def resolve_duplicate_comment_ids(df: pd.DataFrame,
                                  id_col: str = "评论ID",
                                  content_col: str = "评论内容") -> pd.DataFrame:
    """
    检查相同 评论ID 是否对应多个 row。
    情况1：同一 评论ID 的 评论内容 全部一致(经过清洗后的一致)：
        - 打印一条说明
        - 只保留一行，其他行删除
    情况2：同一 评论ID 的 评论内容 存在不一致：
        - 打印⚠警告并列出该ID下的所有不同版本内容
        - 暂不自动删这些行（保留原始多版本，后续人工判断）

    返回: 处理后的 df（已去除 情况1 的纯重复行）。
    约定：不做防御性编程，假设列存在且为字符串 dtype。
    """
    # 我们先复制，避免原 df 原地修改导致调用方困惑
    out = df.copy()

    # 先按 评论ID 分组，聚合每组的唯一 评论内容
    grp = out.groupby(id_col)[content_col].agg(lambda x: list(pd.unique(x)))

    # 找到两类ID
    ids_all_same = grp[grp.map(lambda uniq_list: len(uniq_list) == 1)]
    ids_conflict = grp[grp.map(lambda uniq_list: len(uniq_list) > 1)]

    # --- 情况2：冲突 (同ID多内容) -> 打印警告
    if not ids_conflict.empty:
        print("⚠ 以下 评论ID 映射到多个不同的评论内容，需人工检查：")
        for cid, contents in ids_conflict.items():
            print(f"评论ID={cid}")
            for idx_c, c in enumerate(contents, start=1):
                print(f"    版本{idx_c}: {c}")
            print("-" * 40)

    # --- 情况1：完全相同内容的重复 -> 我们只保留一条
    # 思路：对这些ID对应的行，基于 [评论ID, 评论内容] 做drop_duplicates，其他列随第一条
    if not ids_all_same.empty:
        # mask1: 属于这批“安全可折叠”的ID
        mask_same = out[id_col].isin(ids_all_same.index)
        same_chunk = out.loc[mask_same]

        # 在 same_chunk 内部，按 (评论ID, 评论内容) 去重
        dedup_same_chunk = same_chunk.drop_duplicates(subset=id_col, keep="first")

        # mask2: 其他行（未触发情况1，或者是冲突ID等）
        mask_other = ~mask_same
        other_chunk = out.loc[mask_other]

        # 合并回来，保持原index顺序的相对稳定性：先排序索引再concat
        out = (pd.concat([dedup_same_chunk, other_chunk], axis=0)
                 .sort_index(kind="stable"))

        # 打个summary方便我们知道删了多少
        removed_rows = len(same_chunk) - len(dedup_same_chunk)
        print(f"已合并相同 评论ID & 相同 评论内容 的重复行，共移除 {removed_rows} 行。")
    else:
        print("未发现可以安全折叠的(评论ID相同 & 评论内容一致)重复行。")

    return out

def _ensure_cols_exist(df, cols):
    miss = set(cols) - set(df.columns)
    if miss:
        raise ValueError(f"CSV缺少必要列: {miss}")

def find_dup_by_user_and_content(df, min_count: int = 5) -> pd.DataFrame:
    """同一 用户ID + 评论内容 视作完全一致；返回包含重复组的行，并打印重复次数>=min_count的组。"""
    key = ["用户ID", "评论内容"]
    _ensure_cols_exist(df, key)
    dup_mask = df.duplicated(subset=key, keep=False)
    dup = df.loc[dup_mask].copy()
    if dup.empty:
        print("无完全一致的重复评论。")
        return dup
    dup["重复组大小"] = dup.groupby(key)["评论ID" if "评论ID" in df.columns else "评论内容"].transform("size")
    dup5 = dup[dup["重复组大小"] >= min_count].copy()
    if dup5.empty:
        print(f"无重复次数≥{min_count}的完全一致评论。")
    else:
        # 代表行：按组大小降序、点赞数（若存在）降序、用户ID升序，取每组第一条
        dup5["_like"] = pd.to_numeric(dup5["点赞数"], errors="coerce").fillna(0)
        rep = (dup5.sort_values(["重复组大小","_like","用户ID"], ascending=[False, False, True])
                    .drop_duplicates(subset=key, keep="first"))
        rep = rep.drop(columns=["_like"])

        cols_to_show = [c for c in ["用户ID","评论内容","评论ID","点赞数","评论时间","重复组大小"] if c in rep.columns]
        print(rep.loc[:, cols_to_show].to_string(index=False))
    return dup

def dedupe_keep_max_like(df) -> pd.DataFrame:
    """对完全一致评论（用户ID+评论内容）去重，只保留点赞数最多的一条。"""
    _ensure_cols_exist(df, ["用户ID", "评论内容", "点赞数"])
    tmp = df.copy()
    tmp["_like"] = pd.to_numeric(tmp["点赞数"], errors="coerce").fillna(0)
    # 每组取点赞数最大的索引；若并列，取首个出现
    idx = tmp.groupby(["用户ID","评论内容"])["_like"].idxmax()
    dedup = tmp.loc[idx].drop(columns=["_like"]).sort_index()
    return dedup

def print_frequent_contents_after_clean(dedup_df, min_count: int = 5):
    """在去重后的数据上，仅按评论内容统计，打印出现次数≥min_count的评论内容及次数。"""
    _ensure_cols_exist(dedup_df, ["评论内容"])
    counts = dedup_df["评论内容"].value_counts()
    hot = counts[counts >= min_count].rename_axis("评论内容").reset_index(name="出现次数")
    if hot.empty:
        print(f"清洗后无出现次数≥{min_count}的评论内容。")
    else:
        display(hot.style.set_properties(subset=["评论内容"]))
        # print(hot)
    return hot






CSV_FILENAME = "./data/bilibili_video_data_final.csv"
CHARA_FILENAME='./data/character_info.csv'
video_df = pd.read_csv(CSV_FILENAME)
check_video_index_list=video_df.index
all_nonstd_symbols = set()
bad_idx=[]
for i in check_video_index_list:
    video_bvid = video_df['bvid'][i]  # example path provided by you
    comment_filename = f'./data/{video_bvid}_comment.csv'
    preprocessed_filename = f'./data/{video_bvid}_comment_preprocessed.csv'
    # Read
    if os.path.exists(comment_filename):
        # print(f'{video_bvid} 存在')
        df = pd.read_csv(comment_filename, encoding="utf-8-sig", dtype=str)
        df = resolve_duplicate_comment_ids(df, id_col="评论ID", content_col="评论内容")
        rate = len(df)/video_df['comment_count'][i]
        if rate<0.95:
            print(f'记录长度{video_df['comment_count'][i]}， 处理后长度{len(df)}，比值为{rate}')
            bad_idx.append(i)
    else:
        print(f'{video_bvid} 不存在')

    df['评论内容'] = clean_comment_series(df['评论内容'])


    # ==== 调用 ====
    # 1) 找到完全一致的重复，并打印重复次数≥5的评论
    # dup_df = find_dup_by_user_and_content(df, min_count=5)
    # print(len(dup_df))

    # 2) 基于全部评论去重：对(用户ID, 评论内容)相同的，只保留点赞数最大的    
    df = df[df["评论内容"].fillna("").str.strip().ne("")].copy()
    dedup_df = dedupe_keep_max_like(df)


    # 3) 在去重后的数据上，打印出现次数≥5的评论内容（不要求用户ID相同）
    # hot = print_frequent_contents_after_clean(dedup_df, min_count=5)

    dedup_df.to_csv(preprocessed_filename, index=False, encoding='utf-8-sig')


In [ ]:
# Minimal, fail-fast Python to compute time-to-banner (positive before, negative after)
import pandas as pd
from pathlib import Path

# --- Inputs (assumed to exist) ---
VIDEO_FILENAME = "bilibili_video_data_final.csv"
CHARA_FILENAME = 'character_info.csv'
# --- Load ---
chars = pd.read_csv(CHARA_FILENAME)
videos = pd.read_csv(VIDEO_FILENAME)

# --- Parse dates ---
# Start date has canonical time 04:00 (local-naive); enforce by normalizing date and adding 4h.
chars["start_date"] = pd.to_datetime(chars["start_date"], errors="raise")
chars["start_datetime"] = chars["start_date"].dt.normalize() + pd.Timedelta(hours=4)

# Publish date may already include time; if it's date-only it becomes 00:00 same day.
videos["publish_datetime"] = pd.to_datetime(videos["publish_date"], errors="raise")

# --- Merge (many videos to one character) ---
df = videos.merge(
    chars[["character", "start_datetime"]],
    on="character",
    how="left",
    validate="many_to_one"
)

# --- Compute distance: positive if before banner, negative if after ---
delta = df["start_datetime"] - df["publish_datetime"]
df["time_to_banner_hours"] = delta.dt.total_seconds() / 3600.0  # optional: hours

# --- Save and preview ---
OUT_PATH = "./data/bilibili_video_with_time_to_banner.csv"
df.to_csv(OUT_PATH, index=False, encoding='utf-8-sig')




In [ ]:
import pandas as pd

# 1) 载入视频元数据
videos = pd.read_csv(
    "./data/bilibili_video_with_time_to_banner.csv",
    parse_dates=["publish_datetime"]
)

# 2) 逐视频读取评论、增列并收集
dfs = []
for _, row in videos[["bvid", "character", "video_type", "publish_datetime"]].iterrows():
    bvid = row["bvid"]
    pub_time = row["publish_datetime"]

    cmt = pd.read_csv(
        f"./data/{bvid}_comment_preprocessed.csv",
        parse_dates=["评论时间"]
    )

    # 只保留必要列
    cmt = cmt[[
        "上级评论ID", "评论ID", "用户ID",
        "评论内容", "评论时间",
        "回复数", "点赞数"
    ]]

    # 增加三列：character、video_type、是否在24小时内
    cmt["character"] = row["character"]
    cmt["video_type"] = row["video_type"]
    cmt["within_24h"] = (
        (cmt["评论时间"] >= pub_time) &
        (cmt["评论时间"] <  pub_time + pd.Timedelta(hours=24))
    ).astype(int)

    dfs.append(cmt)

# 3) 合并所有评论并输出
merged = pd.concat(dfs, ignore_index=True)

# 可选：落盘（只包含重要列 + 新增列）
merged.to_csv("./data/all_comments_merged.csv", index=False)